In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

%matplotlib inline

In [2]:
links = pd.read_csv('../lecture-1/links.csv')
movies = pd.read_csv('../lecture-1/movies.csv')
ratings = pd.read_csv('../lecture-1/ratings.csv')
tags = pd.read_csv('../lecture-1/tags.csv')

In [3]:
rm = ratings.merge(movies,on='movieId',how='left')
rm.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [4]:
rm.groupby('userId')['rating'].count().sort_values(ascending=False)

userId
414    2698
599    2478
474    2108
448    1864
274    1346
610    1302
68     1260
380    1218
606    1115
288    1055
249    1046
387    1027
182     977
307     975
603     943
298     939
177     904
318     879
232     862
480     836
608     831
600     763
483     728
590     728
105     722
19      703
305     677
489     648
111     646
438     635
       ... 
531      22
157      21
324      21
364      21
598      21
245      21
549      21
293      21
439      21
49       21
37       21
507      21
281      21
547      21
87       21
26       21
257      20
189      20
595      20
53       20
320      20
278      20
207      20
431      20
576      20
569      20
194      20
147      20
406      20
442      20
Name: rating, Length: 610, dtype: int64

In [5]:
userfilms = rm[rm['userId']==414][['movieId','rating']]
userfilms.head()

,movieId,rating
62294,1,4.0
62295,2,3.0
62296,3,4.0
62297,5,2.0
62298,6,3.0


In [6]:
tagsfilm = movies.merge(tags,on='movieId',how='left').groupby(['movieId'])['tag'].apply(list).to_frame().reset_index(level=0)
tagsfilm['tag'] = tagsfilm['tag'].apply(lambda x: str(x).replace('[','').replace(']','')
                                        .replace(',',' ').replace("'",''))

tagsfilm = tagsfilm.merge(movies,on='movieId',how='left')[['movieId','tag','genres']]

tagsfilm['genres']= tagsfilm['genres'].apply(lambda x: ' '.join(str(x).replace(' ', '').replace('-', '').split('|')))
tagsfilm.head()

,movieId,tag,genres
0,1,pixar pixar fun,Adventure Animation Children Comedy Fantasy
1,2,fantasy magic board game Robin Williams game,Adventure Children Fantasy
2,3,moldy old,Comedy Romance
3,4,nan,Comedy Drama Romance
4,5,pregnancy remake,Comedy


In [15]:
df1 = tagsfilm.merge(userfilms,on='movieId',how='left')
df1['rating'].fillna('Не смотрел',inplace=True)
df1.head()

,movieId,tag,genres,rating
0,1,pixar pixar fun,Adventure Animation Children Comedy Fantasy,4
1,2,fantasy magic board game Robin Williams game,Adventure Children Fantasy,3
2,3,moldy old,Comedy Romance,4
3,4,nan,Comedy Drama Romance,Не смотрел
4,5,pregnancy remake,Comedy,2


In [8]:
from sklearn_pandas import DataFrameMapper
from sklearn.feature_extraction.text import TfidfVectorizer

mapper = DataFrameMapper([
     ('tag', TfidfVectorizer()),
     ('genres', TfidfVectorizer())
 ])

In [9]:
features = mapper.fit_transform(df1)
features = pd.DataFrame(features)
features['movieId'] = df1['movieId']
features['rating'] = df1['rating']
Train = features[features['rating'] != 'Не смотрел']
Val = features[features['rating'] == 'Не смотрел']

In [10]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(Train[Train.columns.difference(['movieId','rating'])], Train['rating'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [11]:
reg.score(Train[Train.columns.difference(['movieId','rating'])], Train['rating'])

0.3313493209266569

In [12]:
predictions = reg.predict(Val[Val.columns.difference(['movieId','rating'])])

In [13]:
Val['predictions'] = predictions
Val['predictions'] = Val['predictions'].astype(int)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
Val[Val['predictions']==5].merge(movies,on='movieId',how='left')[['title','predictions']].sort_values(by='predictions', ascending=False)


,title,predictions
0,Restoration (1995),5
12,Ghosts of the Abyss (2003),5
21,"Invincible Iron Man, The (2007)",5
20,Tekkonkinkreet (Tekkon kinkurîto) (2006),5
19,Kiki's Delivery Service (Majo no takkyûbin) (1...,5
18,Tarnation (2003),5
17,Tokyo Godfathers (2003),5
16,"11'09""01 - September 11 (2002)",5
15,Oklahoma! (1955),5
14,Hannah and Her Sisters (1986),5
